In [5]:
from pydub import AudioSegment
from pydub.utils import which

AudioSegment.converter = which("ffmpeg")


In [8]:
import os
from pydub import AudioSegment

# Path to the directory containing mp3 files
input_dir = "C:\\Users\\jadit\\OneDrive\\Desktop\\SIT\\SEM-6\\PBL\\OnBGGDataset\\gun_sound_v2_dataset"
output_dir = 'Dataset' 

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Convert mp3 to wav
for file in os.listdir(input_dir):
    if file.endswith('.mp3'):
        mp3_file = os.path.join(input_dir, file)
        wav_file = os.path.join(output_dir, os.path.splitext(file)[0] + '.wav')

        audio = AudioSegment.from_mp3(mp3_file)
        audio.export(wav_file, format='wav')
        print(f"Converted: {file} -> {wav_file}")

print("Conversion complete!")


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:
import os
import numpy as np
import librosa
import pandas as pd

def compute_tdoa(audio, sr):
    """
    Computes TDOA using cross-correlation (assumes two-channel audio).
    """
    if audio.ndim < 2:
        return None  # Return None if mono audio

    channel_1 = audio[0, :]
    channel_2 = audio[1, :]

    correlation = np.correlate(channel_1, channel_2, mode="full")
    delay = np.argmax(correlation) - (len(channel_1) - 1)
    
    tdoa = delay / sr  # Convert to time difference (seconds)
    return tdoa

def extract_features(file_path):
    """
    Extracts features from a given audio file.
    """
    y, sr = librosa.load(file_path, sr=None, mono=False)  # Keep stereo/multi-channel

    if y.ndim == 1:  # Convert mono to multi-channel format for consistency
        y = np.expand_dims(y, axis=0)

    # Compute TDOA (only if multi-channel)
    tdoa = compute_tdoa(y, sr) if y.shape[0] > 1 else None
    
    # Convert to mono for spectral feature extraction
    y_mono = librosa.to_mono(y)
    
    # Zero Crossing Rate
    zcr = np.mean(librosa.feature.zero_crossing_rate(y_mono))
    
    # Spectral Features
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y_mono, sr=sr))
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y_mono, sr=sr))
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y_mono, sr=sr))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y_mono, sr=sr))

    # Entropy of Energy
    energy = np.square(y_mono)
    entropy_energy = -np.sum(energy * np.log2(energy + 1e-10))  # Avoid log(0)
    
    # Short-Time Energy (STE)
    frame_length = 2048
    hop_length = 512
    ste = np.array([
        np.sum(np.square(y_mono[i:i + frame_length]))
        for i in range(0, len(y_mono), hop_length)
    ])
    short_time_energy = np.mean(ste)
    
    # MFCCs (1st to 5th)
    mfccs = librosa.feature.mfcc(y=y_mono, sr=sr, n_mfcc=5)
    mfcc_1, mfcc_2, mfcc_3, mfcc_4, mfcc_5 = np.mean(mfccs, axis=1)

    return {
        "Filename": os.path.basename(file_path),
        "TDOA": tdoa,
        "Zero_Crossing_Rate": zcr,
        "Spectral_Centroid": spectral_centroid,
        "Spectral_Bandwidth": spectral_bandwidth,
        "Spectral_Contrast": spectral_contrast,
        "Spectral_Rolloff": spectral_rolloff,
        "Entropy_Energy": entropy_energy,
        "Short_Time_Energy": short_time_energy,
        "MFCC_1": mfcc_1,
        "MFCC_2": mfcc_2,
        "MFCC_3": mfcc_3,
        "MFCC_4": mfcc_4,
        "MFCC_5": mfcc_5,
    }

def process_audio_folder(folder_path, output_csv):
    """
    Extracts features from all WAV files in the folder and saves them to a CSV.
    """
    data = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".wav"):
            file_path = os.path.join(folder_path, file_name)
            print(f"Processing: {file_name}")
            features = extract_features(file_path)
            data.append(features)

    # Convert to DataFrame
    df = pd.DataFrame(data)

    # Save to CSV
    df.to_csv(output_csv, index=False)
    print(f"Saved results to {output_csv}")

# Example Usage
folder_path = "your_wav_folder"  # Replace with the actual folder path
output_csv = "audio_features.csv"
process_audio_folder(folder_path, output_csv)
